This notebook showcase the structure factor of push and gravity equilibrium point process and their pair correlations function

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 24
mpl.rcParams['ytick.labelsize'] = 24
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=16)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [ ]:
# to delete this cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
from structure_factor.point_pattern import PointPattern

# d=2

In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BallWindow

intensity=1/np.pi
point_process = HomogeneousPoissonPointProcess(intensity=intensity)
radius=200
d=2
window = BallWindow(center=[0]*d, radius=radius)
point_pattern = point_process.generate_point_pattern(window=window)
poisson = point_pattern.points

ax = point_pattern.plot()
ax.set_aspect("equal", "box")
plt.tight_layout(pad=1)

Pushed Poisson point process with epsilon = volume_basin/100 and time = 9

In [ ]:
from GPPY.gravity_point_process import GravityPointProcess
gpp_process = GravityPointProcess(point_pattern)

volume_basin = gpp_process.allocation_basin_volume
epsilon = volume_basin**(1/d)/100
start_time = time.time()
if __name__ == "__main__":
    hpp_pattern = gpp_process.pushed_point_pattern(epsilon=volume_basin/100, stop_time=9)
print("--- %s seconds ---" % (time.time() - start_time))
hpp_pattern.plot(window= BallWindow(center=[0,0], radius=40))

In [ ]:
gpp_process = GravityPointProcess(point_pattern)
point_pattern.plot()
for t in range(1,30):
    hpp_pattern_t = gpp_process.pushed_point_pattern(epsilon=epsilon, stop_time=t)
    print(t)
    hpp_pattern_t.plot()
    plt.show()
    

In [ ]:
fig, axis = plt.subplots(1, 3, figsize=(26, 8))
axis[0].scatter(hpp_process[:,0],  hpp_process[:,1], c="b", s=1)
axis[0].scatter(poisson[:,0],  poisson[:,1], c="k", s=1)
axis[1].scatter(poisson[:,0],  poisson[:,1], c="k", s=1)
axis[2].scatter(hpp_process[:,0],  hpp_process[:,1], c="b", s=1)
axis[0].title.set_text("Poisson and Pushed")
axis[1].title.set_text("Poisson")
axis[2].title.set_text("Pushed")
plt.show()

Gravity equilibrium Poisson point process with epsilon = volume_basin/40 and stop_time = 4

In [ ]:
from GPPY.gravity_point_process import GravityPointProcess
gpp_process = GravityPointProcess(point_pattern)

volume_basin = gpp_process.allocation_basin_volume
start_time = time.time()
epp_pattern = gpp_process.equilibrium_point_pattern(epsilon=epsilon, stop_time=6)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
epp_pattern.plot(window= BallWindow(center=[0,0], radius=40))

In [ ]:
fig, axis = plt.subplots(1, 3, figsize=(26, 8))
axis[0].scatter(epp_process[:,0],  epp_process[:,1], c="b", s=1)
axis[0].scatter(poisson[:,0],  poisson[:,1], c="k", s=1)
axis[1].scatter(poisson[:,0],  poisson[:,1], c="k", s=1)
axis[2].scatter(epp_process[:,0],  epp_process[:,1], c="b", s=1)
axis[0].title.set_text("Poisson and Equi")
axis[1].title.set_text("Poisson")
axis[2].title.set_text("Equi")
plt.show()

# Hyperuniformity

In [ ]:
# restrict the pattern to a cubic window
from structure_factor.spatial_windows import BoxWindow
L = (point_pattern.window.radius-2)/np.sqrt(2) # side length of the cubic window
print("Restricting the window to a cube of length", L)
bounds = [[-L/2, L/2], [-L/2, L/2]] 
window = BoxWindow(bounds) # create a cubic window
hpp_pattern_box = hpp_pattern.restrict_to_window(window) 
hpp_pattern_box.plot()

epp_pattern_box = epp_pattern.restrict_to_window(window) 
epp_pattern_box.plot()

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_hpp_pattern = StructureFactor(hpp_pattern_box) 
sf_epp_pattern = StructureFactor(epp_pattern_box)

In [ ]:
k_hpp, s_si_hpp = sf_hpp_pattern.scattering_intensity(k_max=8, 
                                            meshgrid_shape=(150, 150),
                                            )
k_epp, s_si_epp = sf_epp_pattern.scattering_intensity(k_max=18, 
                                            meshgrid_shape=(150, 150),
                                            )

In [ ]:
fig = sf_hpp_pattern.plot_non_isotropic_estimator(k_hpp, s_si_hpp, plot_type="all",
                                              bins=100, error_bar=True,
                                              )
fig = sf_epp_pattern.plot_non_isotropic_estimator(k_epp, s_si_epp, plot_type="all",
                                              bins=40, error_bar=True,
                                              )

In [ ]:
# Directly debiased multitapered estimator
from structure_factor.tapers import SineTaper
 
# Wavevectors
k_max = 14 # Threshold on the maximum wavenumber
x = np.linspace(-k_max, k_max, 17**2)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))

# First 4 tapers of the family of sinusoidal tapers
from structure_factor.tapers import multi_sinetaper_grid
tapers = multi_sinetaper_grid(d=2, p_component_max=2)

# Scaled multitapered periodogram
start_time = time.time()
k_hpp, s_ddmt_hpp = sf_hpp_pattern.tapered_estimator(k, tapers=tapers, debiased=True, direct=True)
k_epp, s_ddmt_epp = sf_epp_pattern.tapered_estimator(k, tapers=tapers, debiased=True, direct=True)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
from structure_factor.point_processes import GinibrePointProcess
ginibre = GinibrePointProcess

In [ ]:
# Plot
sf_hpp_pattern.plot_non_isotropic_estimator(k_hpp, s_ddmt_hpp, plot_type="all", 
                                error_bar=True, bins=70, 
                                scale="log", rasterized=True,
                                exact_sf=ginibre.structure_factor,
                                label=r"$\widehat{S}$")
# Plot
sf_epp_pattern.plot_non_isotropic_estimator(k_epp, s_ddmt_epp, plot_type="all", 
                                error_bar=True, bins=70, 
                                scale="log", rasterized=True,
                                exact_sf=ginibre.structure_factor,
                                label=r"$\widehat{S}$")

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_hpp_pattern_radial = StructureFactor(hpp_pattern) 
sf_epp_pattern_radial = StructureFactor(epp_pattern)

In [ ]:
from structure_factor.point_processes import GinibrePointProcess
ginibre = GinibrePointProcess

In [ ]:
# pcf.fv
import structure_factor.pair_correlation_function as pcf

pcf_fv_hpp = pcf.estimate(hpp_pattern, # PointPattern 
                      method="fv", # Estimation method (could be "ppp" or "fv")
                      Kest=dict(rmax=20), # Maximal estimation radius 
                      fv=dict(method="b", spar=0.2) # Correction method and sparsity
                             )

pcf_fv_epp = pcf.estimate(epp_pattern, # PointPattern 
                      method="fv", # Estimation method (could be "ppp" or "fv")
                      Kest=dict(rmax=20), # Maximal estimation radius 
                      fv=dict(method="b", spar=0.2) # Correction method and sparsity
                             )

In [ ]:
# Plot
pcf.plot(pcf_fv_hpp, 
         #exact_pcf=ginibre.pair_correlation_function,
         figsize=(7,6), 
         color=['grey'], 
         style=["."]
        )
# Plot
pcf.plot(pcf_fv_epp, 
         #exact_pcf=ginibre.pair_correlation_function,
         figsize=(7,6), 
         color=['grey'], 
         style=["."]
        )

In [ ]:
z = np.zeros((1, 2))
index = points != z
index

In [ ]:
from gravitypointprocess import gravity_point_process

In [ ]:
import numpy as np
distances = np.linalg.norm(points, axis=1)
distances

In [ ]:
np.atleast_2d(distances).T

In [ ]:
points

In [ ]:
-1.29286787/1.60122821

In [ ]:
tt = np.divide(points, np.atleast_2d(distances).T)
tt

In [ ]:
points.shape[1]

In [ ]:
from structure_factor.spatial_windows import UnitBallWindow
center = np.full(shape=(2), fill_value=0)
UnitBallWindow(center=center).volume

In [ ]:
import numpy as np
a = np.full((7,3), fill_value=1)
a[1] = a[1] + 2
a

In [ ]:
list(range(8))

In [ ]:
1/2 + 1/12 

In [ ]:
7/12

In [ ]:
index[:,1]

In [ ]:
a[index[:,1]]

In [ ]:
ss = np.linalg.norm(points, axis=1)
ss

In [ ]:
mm =  points != x
points[mm[:,1]]

In [ ]:
points[1:10]

In [ ]:
xx